In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)

import tkinter as tk
from tkinter import filedialog
import warnings
warnings.filterwarnings("ignore")

In [ ]:
root = tk.Tk()
root.withdraw()
path = filedialog.askopenfilename()

In [ ]:
df = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\庫存\inventory.xlsx")

In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
results = df[df["中計代號"].str.contains("LSR2|TBR")]

In [ ]:
results.columns

In [ ]:
ref = pd.read_excel(r"C:\Users\kc.hsu\Python workspace\sales_related\ReferenceTable\20221123_PCODE整理表.xlsx")

In [ ]:
ref["吋別"] = ref["規格"].map(lambda x: x.split("R")[-1])

In [ ]:
pcode_size = dict(zip(ref["PCODE"], ref["規格"]))
pcode_pat = dict(zip(ref["PCODE"], ref["花紋"]))
pcode_place = dict(zip(ref["PCODE"], ref["產地"]))
pcode_inch = dict(zip(ref["PCODE"], ref["吋別"]))

In [ ]:
results["規格"] = results["物料號碼"].map(pcode_size)
results["花紋"] = results["物料號碼"].map(pcode_pat)
results["產地"] = results["物料號碼"].map(pcode_place)
results["吋別"] = results["物料號碼"].map(pcode_inch)

In [ ]:
results["產地"] = results["產地"].map(lambda x: "TH" if x in ["BTMT", "TBSC"] else "JP" if x == "BSJ" else x)

In [ ]:
results["胎別"] = results["物料號碼"].map(lambda x: "TBR" if "TBR" in x else "LSR")

In [ ]:
final = results.groupby(['胎別', '規格', '花紋', '產地', '吋別']).sum().stack().reset_index().rename(columns={"level_5": "倉庫別", 0: "數量"})

In [ ]:
final

In [ ]:
final = final[~final["倉庫別"].isin(['REP合計', '全體REP', 'REP販社', '本社', '販社'])]

In [ ]:
final["數量"] = final["數量"].astype(int)

In [ ]:
final

In [ ]:
final.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\庫存\inventory_prepocessed_by_warehouse.xlsx", index=False)